In [ ]:
import random
random.seed(38)
arr = [round(random.uniform(-5, 5), 5) for i in range(100)]
print(arr)

[1.39474, -0.70979, 2.24299, -4.33769, 1.97863, -1.25068, 1.08036, -3.29909, -1.55433, -2.33037, -1.70521, 2.26597, -0.26949, 4.62792, 0.82841, -2.76707, 1.12798, 0.28049, -3.59171, 0.26846, -4.88318, -1.25593, 0.36387, 1.86223, 4.59414, 1.14663, -4.75422, -0.24466, -2.49799, 2.47286, 3.93631, -4.1322, 2.7163, -2.0678, -3.1717, -0.46467, -0.81243, -2.12473, -2.36136, -0.66502, 1.89474, -3.36071, -1.27174, -2.60801, 2.28585, -2.51953, 2.39677, -4.71227, 3.55744, -0.72273, 4.34169, 2.96006, 3.42565, -4.69841, 2.48449, 0.25369, 3.86223, 0.66685, -1.52719, 4.04277, 4.94745, -2.79086, -3.542, -0.58329, 1.5757, -3.38586, 0.85705, 0.42934, 2.99043, -3.54643, -3.59494, 0.18462, 2.52281, 0.86928, 2.4307, -2.36297, -4.67203, 1.89295, 1.47189, -4.72687, -4.14123, 1.98443, 1.66624, -2.91671, -2.38558, -0.81699, -4.79314, -1.99932, -2.60104, -4.58021, 0.24658, 4.41245, -4.22231, 1.9273, 1.14141, -2.14774, 1.4348, 2.73187, -2.39972, -0.90728]


In [ ]:
import cupy as cp
import numpy as np
from multiprocessing import Process, Queue
from queue import Empty

def precompute_next_j(a, n):
    next_j_list = []
    for i in range(n):
        mask = a[i+1:] >= a[i]
        j_list = cp.where(mask)[0] + (i + 1)
        next_j_list.append(j_list.get())
    return next_j_list

def generate_subsequences_iterative(start_i, k, next_j_list, queue):
    stack = [([start_i], 0)]
    while stack:
        current_path, next_j_index = stack[-1]
        last = current_path[-1]
        if len(current_path) == k:
            queue.put(current_path)
            stack.pop()
            continue
        if next_j_index < len(next_j_list[last]):
            j = next_j_list[last][next_j_index]
            stack[-1] = (current_path, next_j_index + 1)
            stack.append((current_path + [j], 0))
        else:
            stack.pop()

def worker(start_indices, k, next_j_list, queue):
    for start_i in start_indices:
        generate_subsequences_iterative(start_i, k, next_j_list, queue)
    queue.put(None)

def non_decreasing_subsequences_iterator(a, k, num_processes=32):
    n = len(a)
    a_cp = cp.array(a, dtype=cp.float32)
    next_j_list = precompute_next_j(a_cp, n)
    queue = Queue(maxsize=1000)
    processes = []
    start_indices_list = np.array_split(range(n), num_processes)
    for start_indices in start_indices_list:
        p = Process(target=worker, args=(start_indices, k, next_j_list, queue))
        p.start()
        processes.append(p)
    finished_processes = 0
    while finished_processes < num_processes:
        try:
            item = queue.get(timeout=1)
            if item is None:
                finished_processes += 1
            else:
                yield item
        except Empty:
            continue
    for p in processes:
        p.join()

In [ ]:
import bisect

def lnds(nums):
    tails = []
    for num in nums:
        idx = bisect.bisect_right(tails, num)
        if idx == len(tails):
            tails.append(num)
        else:
            tails[idx] = num
    return len(tails)

k = lnds(arr)
print(k)

16


In [ ]:
def float_value_iterator(array, k):
    base_iterator = non_decreasing_subsequences_iterator(array, k)
    for indices in base_iterator:
        yield [array[i] for i in indices]

In [ ]:
iterator = iter(float_value_iterator(arr, k))
while True:
  try:
    print(next(iterator))
  except StopIteration: break

[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.82841, 1.12798, 1.86223, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.94745]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.82841, 1.12798, 1.86223, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.41245]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.82841, 1.12798, 1.14663, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.94745]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.82841, 1.12798, 1.14663, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.41245]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.28049, 0.36387, 1.86223, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.94745]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.28049, 0.36387, 1.86223, 1.89474, 2.28585, 2.39677, 2.96006, 3.42565, 3.86223, 4.04277, 4.41245]
[-4.33769, -3.29909, -2.33037, -1.70521, -0.26949, 0.28049, 0.36387, 1.14663, 1.89474, 2.28585, 2.39